# Génération de Coordonnées 3D à partir de SMILES

Ce notebook lit des chaînes SMILES et des propriétés associées (PCE) à partir d'un fichier CSV, génère les coordonnées 3D pour chaque molécule en utilisant RDKit, et sauvegarde les coordonnées ainsi que la valeur de la propriété dans un fichier texte dans un format spécifique.

## Importation des Bibliothèques

Importation des bibliothèques nécessaires :
- `rdkit` : Pour les tâches de chémoinformatique comme l'analyse des SMILES et la génération de coordonnées 3D.
- `numpy` et `pandas` : Pour les opérations numériques et la manipulation de données (lecture de CSV).
- `sys`, `os` : Pour les opérations au niveau du système comme la gestion des chemins de fichiers.
- `argparse` : Pour gérer les arguments de ligne de commande lors de l'exécution en tant que script.

In [2]:
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
import numpy as np
import pandas as pd
import sys
import os
import argparse

## Fonction `generate_coordinates`

Cette fonction prend le chemin du fichier CSV d'entrée et le chemin du fichier texte de sortie comme arguments et effectue les étapes suivantes :
1.  **Vérification du Fichier :** Vérifie si le fichier CSV d'entrée existe.
2.  **Chargement des Données :** Lit les chaînes SMILES et les valeurs de propriété des colonnes spécifiées ('SMILES', 'PCE') dans le fichier CSV en utilisant pandas. 
3.  **Gestion du Fichier de Sortie :** Ouvre le fichier de sortie spécifié en mode écriture.
4.  **Boucle de Traitement des Molécules :** Itère sur chaque chaîne SMILES :
    *   **SMILES vers Molécule :** Convertit la chaîne SMILES en un objet molécule RDKit (`Chem.MolFromSmiles`). Gère les erreurs si la conversion échoue.
    *   **Intégration 3D :**
        *   Tente de générer un conformère 3D en utilisant l'algorithme ETKDGv3 (`AllChem.EmbedMolecule`).
        *   Si ETKDGv3 échoue (retourne -1), il essaie l'intégration avec des coordonnées aléatoires et optimise la géométrie en utilisant le champ de force MMFF (`AllChem.MMFFOptimizeMolecule`).
    *   **Extraction des Coordonnées :** Récupère le conformère 3D généré.
    *   **Formatage de la Sortie :** Écrit les données dans le fichier de sortie au format souhaité :
        *   Une ligne d'identifiant unique (par exemple, `pce_0`).
        *   Le nombre d'atomes.
        *   Des lignes pour chaque atome : `SymboleAtome X Y Z`.
        *   La valeur de la propriété correspondante.
        *   Deux lignes vides comme séparateur avant la molécule suivante.
    *   **Gestion des Erreurs :** Capture les exceptions lors du traitement d'une seule molécule et affiche un message d'erreur, permettant au script de continuer avec la molécule suivante.
5.  **Fermeture du Fichier :** Ferme le fichier de sortie.
6.  **Message de Fin :** Affiche un message indiquant que le fichier de sortie a été généré.

In [ ]:
def generate_coordinates(filepath, output_file):
    
    if not os.path.exists(filepath):
        raise FileNotFoundError(f"Le fichier {filepath} n'existe pas.")

    df = pd.read_csv(file_path)
    smiles_list = df["SMILES"].tolist()[:2] # Limiter à 2 molécules pour la démo mais on peut l'élargir si le gpu est plus puissant
    
    try:
        property = df["PCE"].tolist()[:2]
    except KeyError:
        property = ""
    
    file = open(output_file, "w", encoding="utf-8")
    
    for ind, smiles in enumerate(smiles_list):
        print(f"Processing molecule {ind + 1}...")
        
        try:
            mol = Chem.MolFromSmiles(smiles)
            
            if mol is None:
                print(f"Erreur : Impossible de convertir SMILES index {ind}")
                continue
        
            m3d = mol
            params = AllChem.ETKDGv3()
            E = AllChem.EmbedMolecule(m3d, params=params)
        
            if E == -1:
                    E = AllChem.EmbedMolecule(m3d, useRandomCoords=True, ignoreSmoothingFailures=True)
                    AllChem.MMFFOptimizeMolecule(m3d, maxIters=10000)
            
            
            conformer = m3d.GetConformer()
            file.write('pce_'+ str(ind)+'\n')
            # Écriture des résultats dans le fichier
            for i in range(m3d.GetNumAtoms()):
                pos = conformer.GetAtomPosition(i)
                atom_symbol = m3d.GetAtomWithIdx(i).GetSymbol()
                file.write(f"{atom_symbol}    {pos.x:.6f}    {pos.y:.6f}     {pos.z:.6f}\n")
            if property == "":
                file.write(f"")
            else :
                file.write(f"{property[ind]}\n")
            file.write("\n\n")
            
        except Exception as e:
                print(f"Erreur lors du traitement de {smiles} : {e}")
                continue
    file.close()
    print(f"Fichier généré : {output_file}")
    

# file_path = r"../datasets/PM/train/train_file.csv"
# output_file = r"../datasets/PM/train/train_m3d.txt"
# generate_coordinates(file_path, output_file)

# file_path = r"../datasets/PM/val/val_file.csv"
# output_file = r"../datasets/PM/val/val_m3d.txt"
# generate_coordinates(file_path, output_file)

# file_path = r"../datasets/PM/test/test_file.csv"
# output_file = r"../datasets/PM/test/test_m3d.txt"
# generate_coordinates(file_path, output_file)

file_path = r"../datasets/data_test/demo.csv"
output_file = r"../datasets/data_test/demo/demo_m3d.txt"
generate_coordinates(file_path, output_file)

Processing molecule 1...
Processing molecule 2...
Fichier généré : ../datasets/data_test/demo/demo_m3d.txt


[10:37:21] Molecule does not have explicit Hs. Consider calling AddHs()
[10:37:21] Molecule does not have explicit Hs. Consider calling AddHs()


## Bloc d'Exécution Principal (apres l'ecriture de la fonction generate_coordinates)

Ce bloc définit comment le script se comporte lorsqu'il est exécuté directement lorsqu'on lui passe des chemins en entrée.
- `file_path` : Le chemin vers le fichier CSV d'entrée (requis).
-  `output_file` : Le chemin pour le fichier texte de sortie (requis).